In [1]:
import numpy as np
import pandas as pd
import os
from math import sqrt
from scipy.optimize import curve_fit
from scipy.optimize import minimize, NonlinearConstraint

## Question#1: Paid Search Bid Optimization

### Part A: Estimate the alpha and beta parameters for each of these four keywords for this firm

In [2]:
os.chdir('C:/Users/athorat/OneDrive - Nice Systems Ltd/00_Amit Thorat Data/ISB - AMPBA/01_Project/Term6_MCA/Assignment/Assignment#3/Assignment 3')
kw8322228 = pd.read_csv('clicksdatakw8322228.csv')
kw8322392 = pd.read_csv('clicksdatakw8322392.csv')
kw8322393 = pd.read_csv('clicksdatakw8322393.csv')
kw8322445 = pd.read_csv('clicksdatakw8322445.csv')

In [3]:
kw8322228.head()

,Unnamed: 0,bid.value,n.clicks
0,1,5.58,15
1,2,8.04,24
2,3,10.77,25
3,4,13.83,30
4,5,17.33,46


In [4]:
def covertArray(df):
    selected_columns = ['bid.value', 'n.clicks']
    numpyArray = df[selected_columns].values
    
    return numpyArray

In [5]:
kw8322228Array = covertArray(kw8322228)
kw8322392Array = covertArray(kw8322392)
kw8322393Array = covertArray(kw8322393)
kw8322445Array = covertArray(kw8322445)
print(kw8322445Array)

[[  0.53  32.  ]
 [  0.76  60.  ]
 [  1.01  76.  ]
 [  1.3   86.  ]
 [  1.63  97.  ]
 [  2.01 101.  ]
 [  2.47 113.  ]
 [  3.04 131.  ]
 [  3.79 163.  ]]


In [6]:
# Define the function form for nonlinear regression
def regression_function(bid_value, alpha, beta):
    return alpha * (1 - np.exp(-beta * bid_value))

In [7]:
def estimateParameter(arrayData):
    # Extract bid values and corresponding clicks from the data
    bidValues = arrayData[:, 0]
    clicks = arrayData[:, 1]
    
    # Use curve_fit to estimate alpha and beta
    params, covariance = curve_fit(regression_function, bidValues, clicks, p0=[70, 0.05])
    
    return params

In [8]:
def printFunction(keyWord, params):
    # Extract estimated alpha and beta
    alphaEst, betaEst = params
    print(f'Parameter for {keyWord}')
    print("Estimated Alpha:", alphaEst)
    print("Estimated Beta:", betaEst)

In [9]:
paramEstkw8322228 = estimateParameter(kw8322228Array)
paramEstkw8322392 = estimateParameter(kw8322392Array)
paramEstkw8322393 = estimateParameter(kw8322393Array)
paramEstkw8322445 = estimateParameter(kw8322445Array)

In [10]:
printFunction('kw8322228',paramEstkw8322228)
printFunction('kw8322392',paramEstkw8322392)
printFunction('kw8322393',paramEstkw8322393)
printFunction('kw8322445',paramEstkw8322445)

Parameter for kw8322228
Estimated Alpha: 74.09072352155722
Estimated Beta: 0.039449156197999116
Parameter for kw8322392
Estimated Alpha: 156.44022274953446
Estimated Beta: 0.15008213875206047
Parameter for kw8322393
Estimated Alpha: 104.79945548823576
Estimated Beta: 0.07971635659169132
Parameter for kw8322445
Estimated Alpha: 188.1117535211361
Estimated Beta: 0.4322899599504818


### Part B: Estimate the optimal bids for each of the four keywords

In [11]:
kwLtvConvRData = pd.read_excel('hw-kw-ltv-conv.rate-data.xlsx')

In [12]:
kwLtvConvRData.head()

,keyword,ltv,conv.rate
0,kw8322228,354,0.30
1,kw8322392,181,0.32
2,kw8322393,283,0.30
3,kw8322445,107,0.30


In [13]:
# Define the desired column names
newColumnNames = ['keyword', 'ltv', 'convrate']

# Rename the columns
kwLtvConvRData.columns = newColumnNames

In [14]:
def sigmoid(x, alpha, beta):
    return 1 / (1 + np.exp(-beta * (x - alpha)))

In [15]:
# Function to calculate expected revenue
def expectedRevenue(bid, alpha, beta, ltv, conv_rate):
    return (sigmoid(bid, alpha, beta) * conv_rate) * ltv

In [16]:
# Optimal bid estimation
def estimateOptimalBids(params, ltvValues, convRateValues):
    optimalBids = []
    for i in range(len(params)):
        alpha, beta = params[i]
        ltv = ltvValues[i]
        conv_rate = convRateValues[i]

        # Objective function to maximize (negative of expected revenue)
        objective = lambda x: -expectedRevenue(x, alpha, beta, ltv, conv_rate)

        # Initial bid guess
        initialBidGuess = 10.0

        # Use optimization to find optimal bid
        result = minimize(objective, initialBidGuess, bounds=[(0, None)])
        optimalBids.append(result.x[0])

    return optimalBids

In [17]:
# LTV and conversion rate values

ltvColumn = ['ltv']
ltv_values = kwLtvConvRData[ltvColumn].values

convrateColumn = ['convrate']
conv_rate_values = kwLtvConvRData[convrateColumn].values

# Estimate optimal bids for each keyword
optimal_bids = estimateOptimalBids([paramEstkw8322228, 
                                    paramEstkw8322392, 
                                    paramEstkw8322393, 
                                    paramEstkw8322445], 
                                   ltv_values, conv_rate_values)

In [18]:
print("Optimal Bids:", optimal_bids)

Optimal Bids: [405.4340245137232, 10.0, 318.7115696574201, 10.0]


### Part C: Compute the optimal bid amounts and the corresponding expenditures for the keywords

In [19]:
# Budget constraint function
def totalExpenditure(bids):
    return np.sum(bids)

In [20]:
# Budget constraint object
budget = 3000
budgetConstraint = NonlinearConstraint(totalExpenditure, 0, budget)

In [21]:
# Function to maximize (negative of expected total profit)
def objectiveWithBudget(bids, params, ltvValues, convRateValues):
    totalRevenue = 0
    for i in range(len(bids)):
        alpha, beta = params[i]
        ltv = ltvValues[i]
        convRate = convRateValues[i]
        totalRevenue += expectedRevenue(bids[i], alpha, beta, ltv, convRate)
    return -totalRevenue

In [22]:
# Initial bids guess
initialBidsGuess = [10.0, 10.0, 10.0, 10.0]

In [23]:
# Optimization with budget constraint
resultWithBudget = minimize(
    objectiveWithBudget, initialBidsGuess,
    args=([paramEstkw8322228, paramEstkw8322392, paramEstkw8322393, paramEstkw8322445], ltv_values, conv_rate_values),
    constraints=[budgetConstraint],
    method='trust-constr', options={'verbose': 1}
)

`gtol` termination condition is satisfied.
Number of iterations: 24, function evaluations: 90, CG iterations: 22, optimality: 2.56e-09, constraint violation: 0.00e+00, execution time: 0.018 s.


In [24]:
optimalBidsWithBudget = resultWithBudget.x
totalExpenditureWithBudget = totalExpenditure(optimalBidsWithBudget)
totalProfit = -resultWithBudget.fun

In [25]:
print("Optimal Bids with Budget Constraint:", optimalBidsWithBudget)
print("Total Expenditure with Budget Constraint:", totalExpenditureWithBudget)
print("Total Profit:", totalProfit)

Optimal Bids with Budget Constraint: [1510.62176684    9.74648196  385.47607076    9.74648178]
Total Expenditure with Budget Constraint: 1915.5908013436376
Total Profit: 191.09999999961374


## Question#2 Display Advertising Assessment

### Assess for each campaign, also of its true expected volume per exposure or impression (abbreviated as "EVI")

In [104]:
from scipy.stats import beta as beta_distribution
from scipy.stats import t

In [105]:
clicksDS2Data = pd.read_excel('clicks.dataset.2.xlsx')

In [106]:
# Transpose the DataFrame
clicksDS2DataT = clicksDS2Data.transpose()

In [107]:
# Print the transposed DataFrame
print(clicksDS2DataT)

         0          1
ad  clicks  exposures
1       52       1000
2       38       1000
3       51       1000
4       45       1000
5       25       1000


In [108]:
# Define the desired column names
newColumnNames = ['clicks', 'exposures']

# Rename the columns
clicksDS2DataT.columns = newColumnNames

In [109]:
clicksDS2DataT = clicksDS2DataT.drop(clicksDS2DataT.index[0])

In [110]:
print(clicksDS2DataT)

  clicks exposures
1     52      1000
2     38      1000
3     51      1000
4     45      1000
5     25      1000


In [111]:
clicksDS2DataT['s'] = clicksDS2DataT['clicks']
clicksDS2DataT['f'] = clicksDS2DataT['exposures'] - clicksDS2DataT['s']
clicksDS2DataT['betaS'] = clicksDS2DataT['s'] + 1
clicksDS2DataT['betaF'] = clicksDS2DataT['f'] + 1

In [112]:
print(clicksDS2DataT)

  clicks exposures   s    f betaS betaF
1     52      1000  52  948    53   949
2     38      1000  38  962    39   963
3     51      1000  51  949    52   950
4     45      1000  45  955    46   956
5     25      1000  25  975    26   976


In [113]:
# Initialize an empty DataFrame
posteriorAnalysisTCTR = pd.DataFrame()
numOfDraws = 100000

In [114]:
# Loop through rows of the parameters DataFrame
for index in range(len(clicksDS2DataT)):
    
    alphaVal = clicksDS2DataT.iloc[index, clicksDS2DataT.columns.get_loc('betaS')]
    betaVal = clicksDS2DataT.iloc[index, clicksDS2DataT.columns.get_loc('betaF')]
        
    # Generate random samples from a beta distribution
    betaDist = beta_distribution.rvs(a=alphaVal, b=betaVal, size=numOfDraws)
    
    # Create a new column in the DataFrame for each draw
    colName = 'drawTCTR{}'.format(index+1)
    posteriorAnalysisTCTR[colName] = betaDist
    
    # Concatenate the temporary DataFrame to the main DataFrame
    #posteriorAnalysisDS2 = dfTemp

In [115]:
posteriorAnalysisTCTR.tail()

,drawTCTR1,drawTCTR2,drawTCTR3,drawTCTR4,drawTCTR5
99995,0.062741,0.045527,0.054467,0.046656,0.035024
99996,0.037793,0.034797,0.069093,0.041388,0.028112
99997,0.056618,0.032996,0.051284,0.051816,0.030489
99998,0.044989,0.041463,0.056533,0.045056,0.033174
99999,0.054862,0.043098,0.065174,0.037599,0.028003


In [116]:
volumelDS2 = pd.read_excel('volumes.dataset.2.xlsx')

In [117]:
volumelDS2.head()

,cust,ad,volume
0,1,1,32
1,2,1,54
2,3,1,31
3,4,1,24
4,5,1,42


In [118]:
# Count the number of customers for each ad category
customerCountByAd = volumelDS2.groupby('ad')['cust'].count()

In [119]:
print(customerCountByAd)

ad
1    52
2    38
3    51
4    45
5    25
Name: cust, dtype: int64


In [120]:
meanVolumeByAd = volumelDS2.groupby('ad')['volume'].mean()
print(meanVolumeByAd)

ad
1    42.076923
2    57.289474
3    41.803922
4    47.888889
5    90.640000
Name: volume, dtype: float64


In [121]:
# Calculate the standard deviation of volumne for each ad category
volumeStdByAd = volumelDS2.groupby('ad')['volume'].std()
print(volumeStdByAd)

ad
1    10.032977
2    17.396657
3    13.891033
4    13.534841
5    30.686425
Name: volume, dtype: float64


In [122]:
# Create a new DataFrame to store the mean values
statDF = pd.DataFrame({
    'ad': meanVolumeByAd.index, 
    'meanVolume': meanVolumeByAd.values,
    'stdVolume': volumeStdByAd.values,
    'custCount': customerCountByAd.values
})

In [123]:
# Convert 'meanVolume','stdVolume' and 'custCount' columns to numeric types
statDF['meanVolume'] = pd.to_numeric(statDF['meanVolume'])
statDF['stdVolume'] = pd.to_numeric(statDF['stdVolume'])
statDF['custCount'] = pd.to_numeric(statDF['custCount'])

In [124]:
# Calculate the standard error and add it to the DataFrame
statDF['stdErr'] = statDF.apply(lambda row: row['stdVolume'] / sqrt(row['custCount']), axis=1)

In [125]:
# Calculate the bayes posterior and add it to the DataFrame
statDF['bayesPost'] = statDF.apply(lambda row: row['custCount'] + 1, axis=1)

In [126]:
statDF

,ad,meanVolume,stdVolume,custCount,stdErr,bayesPost
0,1,42.076923,10.032977,52,1.391324,53.0
1,2,57.289474,17.396657,38,2.822110,39.0
2,3,41.803922,13.891033,51,1.945134,52.0
3,4,47.888889,13.534841,45,2.017655,46.0
4,5,90.640000,30.686425,25,6.137285,26.0


In [127]:
# Initialize an empty DataFrame
posteriorAnalysisVCTR = pd.DataFrame()

In [128]:
# Loop through rows of the parameters DataFrame
for index in range(len(statDF)):
    
    degreeOfFreedom = statDF.iloc[index, statDF.columns.get_loc('bayesPost')]
        
    # Generate random samples from a beta distribution
    betaDist = t.rvs(df=degreeOfFreedom, size=numOfDraws)
    targPostProb = (betaDist * statDF.iloc[index, statDF.columns.get_loc('stdErr')]) + statDF.iloc[index, statDF.columns.get_loc('meanVolume')]
    # Create a new column in the DataFrame for each draw
    colName1 = 'drawVCTR{}'.format(index+1)
    #colName2 = 'EVI{}'.format(index+1)
    posteriorAnalysisVCTR[colName1] = targPostProb
    #posteriorAnalysisVCTR[colName2] = targPostProb

In [129]:
posteriorAnalysisVCTR.tail()

,drawVCTR1,drawVCTR2,drawVCTR3,drawVCTR4,drawVCTR5
99995,40.643461,57.583224,40.686096,45.526770,96.606603
99996,40.291278,58.398164,41.558638,47.054510,85.248383
99997,41.730456,56.564758,40.641688,47.557335,94.154623
99998,42.271654,53.046423,44.168675,44.430948,89.332277
99999,43.085505,60.279049,43.522164,45.701915,94.941420


In [130]:
# Assuming the index is named 'index' in both dataframes
posteriorAnalysisMergedDF = pd.merge(posteriorAnalysisTCTR, posteriorAnalysisVCTR, left_index=True, right_index=True, how='inner')

In [131]:
posteriorAnalysisMergedDF.tail(10)

,drawTCTR1,drawTCTR2,drawTCTR3,drawTCTR4,drawTCTR5,drawVCTR1,drawVCTR2,drawVCTR3,drawVCTR4,drawVCTR5
99990,0.049774,0.043176,0.065312,0.051647,0.025253,41.357797,61.563313,40.143715,47.455190,90.265085
99991,0.052278,0.042299,0.036431,0.051221,0.025182,43.020684,60.044144,42.240956,44.346843,96.180341
99992,0.053797,0.043108,0.052858,0.052474,0.026515,42.486337,63.380071,42.397545,49.038534,80.060035
99993,0.049260,0.036175,0.045306,0.040681,0.021281,41.550435,53.390988,39.453057,46.432524,88.846043
99994,0.058882,0.039328,0.047143,0.041999,0.024815,42.615063,56.786267,42.310835,47.613247,84.582241
99995,0.062741,0.045527,0.054467,0.046656,0.035024,40.643461,57.583224,40.686096,45.526770,96.606603
99996,0.037793,0.034797,0.069093,0.041388,0.028112,40.291278,58.398164,41.558638,47.054510,85.248383
99997,0.056618,0.032996,0.051284,0.051816,0.030489,41.730456,56.564758,40.641688,47.557335,94.154623
99998,0.044989,0.041463,0.056533,0.045056,0.033174,42.271654,53.046423,44.168675,44.430948,89.332277
99999,0.054862,0.043098,0.065174,0.037599,0.028003,43.085505,60.279049,43.522164,45.701915,94.941420


In [132]:
# Calculate the EVI1, 2, 3, 4, & 5 columns
posteriorAnalysisMergedDF['EVI1'] = posteriorAnalysisMergedDF['drawTCTR1'] * posteriorAnalysisMergedDF['drawVCTR1']
posteriorAnalysisMergedDF['EVI2'] = posteriorAnalysisMergedDF['drawTCTR2'] * posteriorAnalysisMergedDF['drawVCTR2']
posteriorAnalysisMergedDF['EVI3'] = posteriorAnalysisMergedDF['drawTCTR3'] * posteriorAnalysisMergedDF['drawVCTR3']
posteriorAnalysisMergedDF['EVI4'] = posteriorAnalysisMergedDF['drawTCTR4'] * posteriorAnalysisMergedDF['drawVCTR4']
posteriorAnalysisMergedDF['EVI5'] = posteriorAnalysisMergedDF['drawTCTR5'] * posteriorAnalysisMergedDF['drawVCTR5']

In [133]:
posteriorAnalysisMergedDF.tail(10)

,drawTCTR1,drawTCTR2,drawTCTR3,drawTCTR4,drawTCTR5,drawVCTR1,drawVCTR2,drawVCTR3,drawVCTR4,drawVCTR5,EVI1,EVI2,EVI3,EVI4,EVI5
99990,0.049774,0.043176,0.065312,0.051647,0.025253,41.357797,61.563313,40.143715,47.455190,90.265085,2.058545,2.658061,2.621886,2.450920,2.279430
99991,0.052278,0.042299,0.036431,0.051221,0.025182,43.020684,60.044144,42.240956,44.346843,96.180341,2.249016,2.539801,1.538874,2.271483,2.422040
99992,0.053797,0.043108,0.052858,0.052474,0.026515,42.486337,63.380071,42.397545,49.038534,80.060035,2.285645,2.732167,2.241056,2.573244,2.122767
99993,0.049260,0.036175,0.045306,0.040681,0.021281,41.550435,53.390988,39.453057,46.432524,88.846043,2.046780,1.931437,1.787470,1.888907,1.890727
99994,0.058882,0.039328,0.047143,0.041999,0.024815,42.615063,56.786267,42.310835,47.613247,84.582241,2.509259,2.233282,1.994673,1.999697,2.098928
99995,0.062741,0.045527,0.054467,0.046656,0.035024,40.643461,57.583224,40.686096,45.526770,96.606603,2.550014,2.621591,2.216049,2.124101,3.383596
99996,0.037793,0.034797,0.069093,0.041388,0.028112,40.291278,58.398164,41.558638,47.054510,85.248383,1.522714,2.032080,2.871407,1.947478,2.396476
99997,0.056618,0.032996,0.051284,0.051816,0.030489,41.730456,56.564758,40.641688,47.557335,94.154623,2.362674,1.866391,2.084253,2.464232,2.870657
99998,0.044989,0.041463,0.056533,0.045056,0.033174,42.271654,53.046423,44.168675,44.430948,89.332277,1.901770,2.199438,2.496989,2.001894,2.963541
99999,0.054862,0.043098,0.065174,0.037599,0.028003,43.085505,60.279049,43.522164,45.701915,94.941420,2.363775,2.597911,2.836517,1.718363,2.658613


In [134]:
# Identify the maximum value among draws1, draws2, draws3 for each row
posteriorAnalysisMergedDF['EVImax'] = posteriorAnalysisMergedDF[['EVI1', 'EVI2', 'EVI3', 'EVI4', 'EVI5']].max(axis=1)

In [135]:
posteriorAnalysisMergedDF['isEVImax1'] = 0
posteriorAnalysisMergedDF['isEVImax2'] = 0
posteriorAnalysisMergedDF['isEVImax3'] = 0
posteriorAnalysisMergedDF['isEVImax4'] = 0
posteriorAnalysisMergedDF['isEVImax5'] = 0

for idx, row in posteriorAnalysisMergedDF.iterrows():
    if row['EVI1'] == row['EVImax']:
        posteriorAnalysisMergedDF.at[idx, 'isEVImax1'] = 1
    elif row['EVI2'] == row['EVImax']:
        posteriorAnalysisMergedDF.at[idx, 'isEVImax2'] = 1
    elif row['EVI3'] == row['EVImax']:
        posteriorAnalysisMergedDF.at[idx, 'isEVImax3'] = 1
    elif row['EVI4'] == row['EVImax']:
        posteriorAnalysisMergedDF.at[idx, 'isEVImax4'] = 1
    elif row['EVI5'] == row['EVImax']:
        posteriorAnalysisMergedDF.at[idx, 'isEVImax5'] = 1

In [136]:
posteriorAnalysisMergedDF.tail(10)

,drawTCTR1,drawTCTR2,drawTCTR3,drawTCTR4,drawTCTR5,drawVCTR1,drawVCTR2,drawVCTR3,drawVCTR4,drawVCTR5,...,EVI2,EVI3,EVI4,EVI5,EVImax,isEVImax1,isEVImax2,isEVImax3,isEVImax4,isEVImax5
99990,0.049774,0.043176,0.065312,0.051647,0.025253,41.357797,61.563313,40.143715,47.455190,90.265085,...,2.658061,2.621886,2.450920,2.279430,2.658061,0,1,0,0,0
99991,0.052278,0.042299,0.036431,0.051221,0.025182,43.020684,60.044144,42.240956,44.346843,96.180341,...,2.539801,1.538874,2.271483,2.422040,2.539801,0,1,0,0,0
99992,0.053797,0.043108,0.052858,0.052474,0.026515,42.486337,63.380071,42.397545,49.038534,80.060035,...,2.732167,2.241056,2.573244,2.122767,2.732167,0,1,0,0,0
99993,0.049260,0.036175,0.045306,0.040681,0.021281,41.550435,53.390988,39.453057,46.432524,88.846043,...,1.931437,1.787470,1.888907,1.890727,2.046780,1,0,0,0,0
99994,0.058882,0.039328,0.047143,0.041999,0.024815,42.615063,56.786267,42.310835,47.613247,84.582241,...,2.233282,1.994673,1.999697,2.098928,2.509259,1,0,0,0,0
99995,0.062741,0.045527,0.054467,0.046656,0.035024,40.643461,57.583224,40.686096,45.526770,96.606603,...,2.621591,2.216049,2.124101,3.383596,3.383596,0,0,0,0,1
99996,0.037793,0.034797,0.069093,0.041388,0.028112,40.291278,58.398164,41.558638,47.054510,85.248383,...,2.032080,2.871407,1.947478,2.396476,2.871407,0,0,1,0,0
99997,0.056618,0.032996,0.051284,0.051816,0.030489,41.730456,56.564758,40.641688,47.557335,94.154623,...,1.866391,2.084253,2.464232,2.870657,2.870657,0,0,0,0,1
99998,0.044989,0.041463,0.056533,0.045056,0.033174,42.271654,53.046423,44.168675,44.430948,89.332277,...,2.199438,2.496989,2.001894,2.963541,2.963541,0,0,0,0,1
99999,0.054862,0.043098,0.065174,0.037599,0.028003,43.085505,60.279049,43.522164,45.701915,94.941420,...,2.597911,2.836517,1.718363,2.658613,2.836517,0,0,1,0,0


In [137]:
# Calculate the average of EVI1, 2, 3, 4, & 5 columns
avgEVI1 = posteriorAnalysisMergedDF['isEVImax1'].mean() * 100
avgEVI2 = posteriorAnalysisMergedDF['isEVImax2'].mean() * 100
avgEVI3 = posteriorAnalysisMergedDF['isEVImax3'].mean() * 100
avgEVI4 = posteriorAnalysisMergedDF['isEVImax4'].mean() * 100
avgEVI5 = posteriorAnalysisMergedDF['isEVImax5'].mean() * 100

In [138]:
# Print the results
print("Average EVI1 as percentage:", avgEVI1)
print("Average EVI2 as percentage:", avgEVI2)
print("Average EVI3 as percentage:", avgEVI3)
print("Average EVI4 as percentage:", avgEVI4)
print("Average EVI5 as percentage:", avgEVI5)

Average EVI1 as percentage: 16.74
Average EVI2 as percentage: 19.855
Average EVI3 as percentage: 13.279
Average EVI4 as percentage: 16.072
Average EVI5 as percentage: 34.054


### First task: Computing the bayesian posterior probability that a certain campaign's true CTR is the highest

In [141]:
import numpy as np
from scipy.stats import beta as beta_dist

In [142]:
drawNumbers = range(1,100001)

In [143]:
# Create a new DataFrame with drawnumber as the index
posteriorAnalysis = pd.DataFrame(index=drawNumbers)

In [144]:
posteriorAnalysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 1 to 100000
Empty DataFrame


In [145]:
# Set a seed for reproducibility
np.random.seed(42)

In [146]:
# Loop through rows of the parameters DataFrame
for index in range(len(clicksDS2DataT)):
       
    alphaVal = clicksDS2DataT.iloc[index, clicksDS2DataT.columns.get_loc('betaS')]
    betaVal = clicksDS2DataT.iloc[index, clicksDS2DataT.columns.get_loc('betaF')]
    
    # Generate random samples from a beta distribution
    randomValues = np.random.rand(numOfDraws)
        
    # Generate random samples from a beta distribution
    betaDist = beta_dist.ppf(randomValues, a=alphaVal, b=betaVal)
    
    # Create a new column in the DataFrame for each draw
    colName = 'draws{}'.format(index+1)
    posteriorAnalysis[colName] = betaDist

In [147]:
posteriorAnalysis.tail(10)

,draws1,draws2,draws3,draws4,draws5
99991,0.058320,0.046936,0.058437,0.052228,0.021873
99992,0.044867,0.044962,0.046833,0.038462,0.030511
99993,0.063770,0.034715,0.057118,0.048771,0.028422
99994,0.051069,0.033680,0.052575,0.043817,0.029404
99995,0.060028,0.029202,0.058250,0.050061,0.035656
99996,0.058537,0.040539,0.042869,0.050192,0.027824
99997,0.058201,0.040656,0.053867,0.041729,0.038855
99998,0.055847,0.033554,0.053886,0.046873,0.025480
99999,0.052587,0.041059,0.040766,0.054278,0.027817
100000,0.050649,0.023240,0.055430,0.045646,0.027278


In [148]:
# Identify the maximum value among draws1, draws2, draws3 for each row
posteriorAnalysis['max'] = posteriorAnalysis[['draws1', 'draws2', 'draws3','draws4','draws5']].max(axis=1)

In [149]:
print(posteriorAnalysis.tail(10))

          draws1    draws2    draws3    draws4    draws5       max
99991   0.058320  0.046936  0.058437  0.052228  0.021873  0.058437
99992   0.044867  0.044962  0.046833  0.038462  0.030511  0.046833
99993   0.063770  0.034715  0.057118  0.048771  0.028422  0.063770
99994   0.051069  0.033680  0.052575  0.043817  0.029404  0.052575
99995   0.060028  0.029202  0.058250  0.050061  0.035656  0.060028
99996   0.058537  0.040539  0.042869  0.050192  0.027824  0.058537
99997   0.058201  0.040656  0.053867  0.041729  0.038855  0.058201
99998   0.055847  0.033554  0.053886  0.046873  0.025480  0.055847
99999   0.052587  0.041059  0.040766  0.054278  0.027817  0.054278
100000  0.050649  0.023240  0.055430  0.045646  0.027278  0.055430


In [150]:
posteriorAnalysis['ismax1'] = 0
posteriorAnalysis['ismax2'] = 0
posteriorAnalysis['ismax3'] = 0
posteriorAnalysis['ismax4'] = 0
posteriorAnalysis['ismax5'] = 0

for idx, row in posteriorAnalysis.iterrows():
    if row['draws1'] == row['max']:
        posteriorAnalysis.at[idx, 'ismax1'] = 1
    elif row['draws2'] == row['max']:
        posteriorAnalysis.at[idx, 'ismax2'] = 1
    elif row['draws3'] == row['max']:
        posteriorAnalysis.at[idx, 'ismax3'] = 1
    elif row['draws4'] == row['max']:
        posteriorAnalysis.at[idx, 'ismax4'] = 1
    elif row['draws5'] == row['max']:
        posteriorAnalysis.at[idx, 'ismax5'] = 1

In [151]:
posteriorAnalysis.tail(10)

,draws1,draws2,draws3,draws4,draws5,max,ismax1,ismax2,ismax3,ismax4,ismax5
99991,0.058320,0.046936,0.058437,0.052228,0.021873,0.058437,0,0,1,0,0
99992,0.044867,0.044962,0.046833,0.038462,0.030511,0.046833,0,0,1,0,0
99993,0.063770,0.034715,0.057118,0.048771,0.028422,0.063770,1,0,0,0,0
99994,0.051069,0.033680,0.052575,0.043817,0.029404,0.052575,0,0,1,0,0
99995,0.060028,0.029202,0.058250,0.050061,0.035656,0.060028,1,0,0,0,0
99996,0.058537,0.040539,0.042869,0.050192,0.027824,0.058537,1,0,0,0,0
99997,0.058201,0.040656,0.053867,0.041729,0.038855,0.058201,1,0,0,0,0
99998,0.055847,0.033554,0.053886,0.046873,0.025480,0.055847,1,0,0,0,0
99999,0.052587,0.041059,0.040766,0.054278,0.027817,0.054278,0,0,0,1,0
100000,0.050649,0.023240,0.055430,0.045646,0.027278,0.055430,0,0,1,0,0


In [152]:
print(posteriorAnalysis.head(10))

      draws1    draws2    draws3    draws4    draws5       max  ismax1  \
1   0.050372  0.039869  0.047678  0.039278  0.032201  0.050372       1   
2   0.065061  0.039029  0.050876  0.037503  0.023312  0.065061       1   
3   0.057075  0.036333  0.043103  0.038783  0.018335  0.057075       1   
4   0.054377  0.038512  0.050670  0.045169  0.033265  0.054377       1   
5   0.045772  0.036554  0.045999  0.053936  0.024841  0.053936       0   
6   0.045771  0.041142  0.047579  0.042399  0.020132  0.047579       0   
7   0.042249  0.044837  0.051216  0.044648  0.023875  0.051216       0   
8   0.060780  0.043653  0.060469  0.034528  0.029716  0.060780       1   
9   0.054423  0.036169  0.052992  0.047296  0.026641  0.054423       1   
10  0.056549  0.038760  0.038199  0.042932  0.034206  0.056549       1   

    ismax2  ismax3  ismax4  ismax5  
1        0       0       0       0  
2        0       0       0       0  
3        0       0       0       0  
4        0       0       0       0  


In [153]:
# Calculate the average of ismax1, ismax2, ismax3 columns
avgIsMax1 = posteriorAnalysis['ismax1'].mean() * 100
avgIsMax2 = posteriorAnalysis['ismax2'].mean() * 100
avgIsMax3 = posteriorAnalysis['ismax3'].mean() * 100
avgIsMax4 = posteriorAnalysis['ismax4'].mean() * 100
avgIsMax5 = posteriorAnalysis['ismax5'].mean() * 100

In [154]:
# Print the results
print("Average ismax1 as percentage:", avgIsMax1)
print("Average ismax2 as percentage:", avgIsMax2)
print("Average ismax3 as percentage:", avgIsMax3)
print("Average ismax4 as percentage:", avgIsMax4)
print("Average ismax5 as percentage:", avgIsMax5)

Average ismax1 as percentage: 46.797
Average ismax2 as percentage: 1.645
Average ismax3 as percentage: 39.784000000000006
Average ismax4 as percentage: 11.770999999999999
Average ismax5 as percentage: 0.003


In [155]:
# Save the DataFrame to a CSV file
posteriorAnalysis.to_csv("DASS1posteriorAnalysis100000.csv", index=False)

print("CSV file 'DASS1posteriorAnalysis100000.csv' created successfully.")

CSV file 'DASS1posteriorAnalysis100000.csv' created successfully.


### Second task: Computing the bayesian posterior probability that a certain campaign's average profit volume is the highest

In [156]:
import numpy as np
from scipy.stats import t

In [157]:
# Create a new DataFrame with drawnumber as the index
highProfPostAna = pd.DataFrame(index=drawNumbers)

In [158]:
highProfPostAna.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 1 to 100000
Empty DataFrame


In [159]:
# Set a seed for reproducibility
np.random.seed(42)

In [160]:
# Loop through rows of the parameters DataFrame
for index in range(len(statDF)):
    
    # Generate a random number from a uniform distribution (equivalent to RAND() in Excel)
    randomNum = np.random.rand(numOfDraws)
    
    degreeofFreedom = statDF.iloc[index, statDF.columns.get_loc('bayesPost')]
    stdError = statDF.iloc[index, statDF.columns.get_loc('stdErr')]
    sampMean = statDF.iloc[index, statDF.columns.get_loc('meanVolume')]
    
    
    # Calculate the inverse of the cumulative distribution function (CDF) for the t-distribution
    tInverse = t.ppf(randomNum,degreeofFreedom)
    # Calculate the final result using the given formula
    result = tInverse * stdError + sampMean
    # Create a new column in the DataFrame for each draw
    colName = 'draws{}'.format(index+1)
    highProfPostAna[colName] = result

In [161]:
highProfPostAna.tail()

,draws1,draws2,draws3,draws4,draws5
99996,43.219036,58.173724,39.194728,49.261567,93.294243
99997,43.155887,58.226821,42.430271,46.658709,105.703872
99998,42.709731,54.806847,42.435595,48.275669,90.449555
99999,42.074986,58.408683,38.499187,50.435112,93.285122
100000,41.685984,48.396444,42.853319,47.901208,92.643781


In [162]:
# Identify the maximum value among draws1, draws2, draws3 for each row
highProfPostAna['max'] = highProfPostAna[['draws1', 'draws2', 'draws3']].max(axis=1)

In [163]:
highProfPostAna['ismax1'] = 0
highProfPostAna['ismax2'] = 0
highProfPostAna['ismax3'] = 0
highProfPostAna['ismax4'] = 0
highProfPostAna['ismax5'] = 0

for idx, row in highProfPostAna.iterrows():
    if row['draws1'] == row['max']:
        highProfPostAna.at[idx, 'ismax1'] = 1
    elif row['draws2'] == row['max']:
        highProfPostAna.at[idx, 'ismax2'] = 1
    elif row['draws3'] == row['max']:
        highProfPostAna.at[idx, 'ismax3'] = 1
    elif row['draws4'] == row['max']:
        highProfPostAna.at[idx, 'ismax4'] = 1
    elif row['draws5'] == row['max']:
        highProfPostAna.at[idx, 'ismax5'] = 1

In [164]:
highProfPostAna.tail(10)

,draws1,draws2,draws3,draws4,draws5,max,ismax1,ismax2,ismax3,ismax4,ismax5
99991,43.178257,60.974408,43.650735,49.850931,85.695508,60.974408,0,1,0,0,0
99992,40.456348,60.127722,40.426648,45.551570,96.409312,60.127722,0,1,0,0,0
99993,44.182409,55.400414,43.303360,48.843749,93.997691,55.400414,0,1,0,0,0
99994,41.771128,54.872000,42.075468,47.330597,95.140285,54.872000,0,1,0,0,0
99995,43.496670,52.402348,43.601706,49.223464,102.157416,52.402348,0,1,0,0,0
99996,43.219036,58.173724,39.194728,49.261567,93.294243,58.173724,0,1,0,0,0
99997,43.155887,58.226821,42.430271,46.658709,105.703872,58.226821,0,1,0,0,0
99998,42.709731,54.806847,42.435595,48.275669,90.449555,54.806847,0,1,0,0,0
99999,42.074986,58.408683,38.499187,50.435112,93.285122,58.408683,0,1,0,0,0
100000,41.685984,48.396444,42.853319,47.901208,92.643781,48.396444,0,1,0,0,0


In [165]:
# Calculate the average of ismax1, ismax2, ismax3 columns
avgIsMax1HPA = highProfPostAna['ismax1'].mean() * 100
avgIsMax2HPA = highProfPostAna['ismax2'].mean() * 100
avgIsMax3HPA = highProfPostAna['ismax3'].mean() * 100
avgIsMax4HPA = highProfPostAna['ismax4'].mean() * 100
avgIsMax5HPA = highProfPostAna['ismax5'].mean() * 100

In [166]:
# Print the results
print("Average ismax1 as percentage for campaign's true CTR is the highest:", avgIsMax1HPA)
print("Average ismax2 as percentage for campaign's true CTR is the highest:", avgIsMax2HPA)
print("Average ismax3 as percentage for campaign's true CTR is the highest:", avgIsMax3HPA)
print("Average ismax4 as percentage for campaign's true CTR is the highest:", avgIsMax4HPA)
print("Average ismax5 as percentage for campaign's true CTR is the highest:", avgIsMax5HPA)

Average ismax1 as percentage for campaign's true CTR is the highest: 0.0
Average ismax2 as percentage for campaign's true CTR is the highest: 99.998
Average ismax3 as percentage for campaign's true CTR is the highest: 0.002
Average ismax4 as percentage for campaign's true CTR is the highest: 0.0
Average ismax5 as percentage for campaign's true CTR is the highest: 0.0


In [167]:
# Save the DataFrame to a CSV file
highProfPostAna.to_csv("DASS1posteriorAnalysishighCTR100000.csv", index=False)

print("CSV file 'DASS1posteriorAnalysishighCTR100000.csv' created successfully.")

CSV file 'DASS1posteriorAnalysishighCTR100000.csv' created successfully.


### Third task: Computing the bayesian posterior probability that a certain campaign's CTR*m is the highest 

In [168]:
# Create a new DataFrame with drawnumber as the index
jointPostAnaCTR = pd.DataFrame(columns=["ctrdraws1", "ctrdraws2", "ctrdraws3", "ctrdraws4", "ctrdraws5", 
                                        "muddraws1", "muddraws2", "muddraws3", "muddraws4", "muddraws5",
                                        "product1", "product2", "product3", "product4", "product5", 
                                        "max", "ismax1", "ismax2", "ismax3", "ismax4", "ismax5"])

In [169]:
jointPostAnaCTR.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 21 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ctrdraws1  0 non-null      object
 1   ctrdraws2  0 non-null      object
 2   ctrdraws3  0 non-null      object
 3   ctrdraws4  0 non-null      object
 4   ctrdraws5  0 non-null      object
 5   muddraws1  0 non-null      object
 6   muddraws2  0 non-null      object
 7   muddraws3  0 non-null      object
 8   muddraws4  0 non-null      object
 9   muddraws5  0 non-null      object
 10  product1   0 non-null      object
 11  product2   0 non-null      object
 12  product3   0 non-null      object
 13  product4   0 non-null      object
 14  product5   0 non-null      object
 15  max        0 non-null      object
 16  ismax1     0 non-null      object
 17  ismax2     0 non-null      object
 18  ismax3     0 non-null      object
 19  ismax4     0 non-null      object
 20  ismax5     0 non-null      object
dtypes: object(

In [170]:
# Display column names for jointPostAnaCTR
print(jointPostAnaCTR.columns)

Index(['ctrdraws1', 'ctrdraws2', 'ctrdraws3', 'ctrdraws4', 'ctrdraws5',
       'muddraws1', 'muddraws2', 'muddraws3', 'muddraws4', 'muddraws5',
       'product1', 'product2', 'product3', 'product4', 'product5', 'max',
       'ismax1', 'ismax2', 'ismax3', 'ismax4', 'ismax5'],
      dtype='object')


In [171]:
# Display column names for posteriorAnalysis
print(posteriorAnalysis.columns)

Index(['draws1', 'draws2', 'draws3', 'draws4', 'draws5', 'max', 'ismax1',
       'ismax2', 'ismax3', 'ismax4', 'ismax5'],
      dtype='object')


In [172]:
# Display column names for highProfPostAna
print(highProfPostAna.columns)

Index(['draws1', 'draws2', 'draws3', 'draws4', 'draws5', 'max', 'ismax1',
       'ismax2', 'ismax3', 'ismax4', 'ismax5'],
      dtype='object')


In [173]:
jointPostAnaCTR['ctrdraws1'] = posteriorAnalysis['draws1']
jointPostAnaCTR['ctrdraws2'] = posteriorAnalysis['draws2']
jointPostAnaCTR['ctrdraws3'] = posteriorAnalysis['draws3']
jointPostAnaCTR['ctrdraws4'] = posteriorAnalysis['draws4']
jointPostAnaCTR['ctrdraws5'] = posteriorAnalysis['draws5']

In [174]:
jointPostAnaCTR['muddraws1'] = highProfPostAna['draws1']
jointPostAnaCTR['muddraws2'] = highProfPostAna['draws2']
jointPostAnaCTR['muddraws3'] = highProfPostAna['draws3']
jointPostAnaCTR['muddraws4'] = highProfPostAna['draws4']
jointPostAnaCTR['muddraws5'] = highProfPostAna['draws5']

In [175]:
jointPostAnaCTR.tail(10)

,ctrdraws1,ctrdraws2,ctrdraws3,ctrdraws4,ctrdraws5,muddraws1,muddraws2,muddraws3,muddraws4,muddraws5,...,product2,product3,product4,product5,max,ismax1,ismax2,ismax3,ismax4,ismax5
99991,0.058320,0.046936,0.058437,0.052228,0.021873,43.178257,60.974408,43.650735,49.850931,85.695508,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99992,0.044867,0.044962,0.046833,0.038462,0.030511,40.456348,60.127722,40.426648,45.551570,96.409312,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99993,0.063770,0.034715,0.057118,0.048771,0.028422,44.182409,55.400414,43.303360,48.843749,93.997691,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99994,0.051069,0.033680,0.052575,0.043817,0.029404,41.771128,54.872000,42.075468,47.330597,95.140285,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99995,0.060028,0.029202,0.058250,0.050061,0.035656,43.496670,52.402348,43.601706,49.223464,102.157416,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99996,0.058537,0.040539,0.042869,0.050192,0.027824,43.219036,58.173724,39.194728,49.261567,93.294243,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99997,0.058201,0.040656,0.053867,0.041729,0.038855,43.155887,58.226821,42.430271,46.658709,105.703872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99998,0.055847,0.033554,0.053886,0.046873,0.025480,42.709731,54.806847,42.435595,48.275669,90.449555,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99999,0.052587,0.041059,0.040766,0.054278,0.027817,42.074986,58.408683,38.499187,50.435112,93.285122,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100000,0.050649,0.023240,0.055430,0.045646,0.027278,41.685984,48.396444,42.853319,47.901208,92.643781,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
# Calculate the m*CTR and add it to the DataFrame
jointPostAnaCTR['product1'] = jointPostAnaCTR.apply(lambda row: row['ctrdraws1'] * row['muddraws1'] , axis=1)
jointPostAnaCTR['product2'] = jointPostAnaCTR.apply(lambda row: row['ctrdraws2'] * row['muddraws2'] , axis=1)
jointPostAnaCTR['product3'] = jointPostAnaCTR.apply(lambda row: row['ctrdraws3'] * row['muddraws3'] , axis=1)
jointPostAnaCTR['product4'] = jointPostAnaCTR.apply(lambda row: row['ctrdraws4'] * row['muddraws4'] , axis=1)
jointPostAnaCTR['product5'] = jointPostAnaCTR.apply(lambda row: row['ctrdraws5'] * row['muddraws5'] , axis=1)

In [177]:
jointPostAnaCTR.head(10)

,ctrdraws1,ctrdraws2,ctrdraws3,ctrdraws4,ctrdraws5,muddraws1,muddraws2,muddraws3,muddraws4,muddraws5,...,product2,product3,product4,product5,max,ismax1,ismax2,ismax3,ismax4,ismax5
1,0.050372,0.039869,0.047678,0.039278,0.032201,41.629583,57.868721,40.677942,45.835369,98.317420,...,2.307169,1.939441,1.800316,3.165946,NaN,NaN,NaN,NaN,NaN,NaN
2,0.065061,0.039029,0.050876,0.037503,0.023312,44.416282,57.481650,41.601095,45.211211,87.660690,...,2.243438,2.116515,1.695558,2.043562,NaN,NaN,NaN,NaN,NaN,NaN
3,0.057075,0.036333,0.043103,0.038783,0.018335,42.943598,56.201981,39.270076,45.663951,80.294360,...,2.041986,1.692649,1.770990,1.472226,NaN,NaN,NaN,NaN,NaN,NaN
4,0.054377,0.038512,0.050670,0.045169,0.033265,42.426329,57.241149,41.542666,47.753978,99.505984,...,2.204485,2.104955,2.157020,3.310097,NaN,NaN,NaN,NaN,NaN,NaN
5,0.045772,0.036554,0.045999,0.053936,0.024841,40.656811,56.309203,40.174936,50.338386,89.645471,...,2.058311,1.848023,2.715074,2.226871,NaN,NaN,NaN,NaN,NaN,NaN
6,0.045771,0.041142,0.047579,0.042399,0.020132,40.656668,58.446198,40.648814,46.877123,83.156307,...,2.404592,1.934049,1.987558,1.674118,NaN,NaN,NaN,NaN,NaN,NaN
7,0.042249,0.044837,0.051216,0.044648,0.023875,39.854703,60.073650,41.696635,47.591841,88.402937,...,2.693513,2.135519,2.124883,2.110648,NaN,NaN,NaN,NaN,NaN,NaN
8,0.060780,0.043653,0.060469,0.034528,0.029716,43.635587,59.558745,44.179939,44.099518,95.499289,...,2.599893,2.671524,1.522677,2.837828,NaN,NaN,NaN,NaN,NaN,NaN
9,0.054423,0.036169,0.052992,0.047296,0.026641,42.435225,56.122063,42.190637,48.403416,91.876664,...,2.029885,2.235770,2.289304,2.447694,NaN,NaN,NaN,NaN,NaN,NaN
10,0.056549,0.038760,0.038199,0.042932,0.034206,42.843735,57.356542,37.600813,47.048771,100.550836,...,2.223112,1.436331,2.019887,3.439392,NaN,NaN,NaN,NaN,NaN,NaN


In [178]:
# Identify the maximum value among draws1, draws2, draws3 for each row
jointPostAnaCTR['max'] = jointPostAnaCTR[['product1', 'product2', 'product3', 'product4', 'product5']].max(axis=1)

In [179]:
jointPostAnaCTR['ismax1'] = 0
jointPostAnaCTR['ismax2'] = 0
jointPostAnaCTR['ismax3'] = 0
jointPostAnaCTR['ismax4'] = 0
jointPostAnaCTR['ismax5'] = 0

for idx, row in jointPostAnaCTR.iterrows():
    if row['product1'] == row['max']:
        jointPostAnaCTR.at[idx, 'ismax1'] = 1
    elif row['product2'] == row['max']:
        jointPostAnaCTR.at[idx, 'ismax2'] = 1
    elif row['product3'] == row['max']:
        jointPostAnaCTR.at[idx, 'ismax3'] = 1
    elif row['product4'] == row['max']:
        jointPostAnaCTR.at[idx, 'ismax4'] = 1
    elif row['product5'] == row['max']:
        jointPostAnaCTR.at[idx, 'ismax5'] = 1

In [180]:
# Calculate the average of ismax1, ismax2, ismax3, ismax4, ismax5 columns
avgIsMax1JPAC = jointPostAnaCTR['ismax1'].mean() * 100
avgIsMax2JPAC = jointPostAnaCTR['ismax2'].mean() * 100
avgIsMax3JPAC = jointPostAnaCTR['ismax3'].mean() * 100
avgIsMax4JPAC = jointPostAnaCTR['ismax4'].mean() * 100
avgIsMax5JPAC = jointPostAnaCTR['ismax5'].mean() * 100

In [181]:
# Print the results
print("Average ismax1 as percentage for campaign's CTR*m is the highest:", avgIsMax1JPAC)
print("Average ismax2 as percentage for campaign's CTR*m is the highest:", avgIsMax2JPAC)
print("Average ismax3 as percentage for campaign's CTR*m is the highest:", avgIsMax3JPAC)
print("Average ismax4 as percentage for campaign's CTR*m is the highest:", avgIsMax4JPAC)
print("Average ismax5 as percentage for campaign's CTR*m is the highest:", avgIsMax5JPAC)

Average ismax1 as percentage for campaign's CTR*m is the highest: 16.307
Average ismax2 as percentage for campaign's CTR*m is the highest: 20.237
Average ismax3 as percentage for campaign's CTR*m is the highest: 14.359
Average ismax4 as percentage for campaign's CTR*m is the highest: 16.176
Average ismax5 as percentage for campaign's CTR*m is the highest: 32.921


In [182]:
# Save the DataFrame to a CSV file
jointPostAnaCTR.to_csv("DASS1posteriorAnalysisjointCTR100000.csv", index=False)

print("CSV file 'DASS1posteriorAnalysisjointCTR100000.csv' created successfully.")

CSV file 'DASS1posteriorAnalysisjointCTR100000.csv' created successfully.
